In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd 
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/FullShotsData.csv')

In [3]:
df.head()

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,year,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
0,378451.0,20,BlockedShot,0.876,0.602,0.019479,Ollie Watkins,h,8865,OpenPlay,2020,Head,14104,Aston Villa,Sheffield United,1,0,2020-09-21 17:00:00,Matthew Cash,Aerial
1,378458.0,54,MissedShots,0.878,0.430,0.031428,Ollie Watkins,h,8865,OpenPlay,2020,Head,14104,Aston Villa,Sheffield United,1,0,2020-09-21 17:00:00,Trézéguet,Cross
2,378464.0,77,MissedShots,0.858,0.679,0.060696,Ollie Watkins,h,8865,OpenPlay,2020,RightFoot,14104,Aston Villa,Sheffield United,1,0,2020-09-21 17:00:00,John McGinn,Pass
3,379954.0,34,MissedShots,0.831,0.655,0.127710,Ollie Watkins,a,8865,OpenPlay,2020,RightFoot,14114,Fulham,Aston Villa,0,3,2020-09-28 17:00:00,Jack Grealish,TakeOn
4,379956.0,46,MissedShots,0.943,0.500,0.445354,Ollie Watkins,a,8865,FromCorner,2020,Head,14114,Fulham,Aston Villa,0,3,2020-09-28 17:00:00,Conor Hourihane,Cross


In [4]:
df['result'].unique()

array(['BlockedShot', 'MissedShots', 'Goal', 'SavedShot', 'ShotOnPost',
       'OwnGoal'], dtype=object)

In [5]:
name = "Mohamed Salah"
salah = df[df['player'] == name]
salah['result'] = salah['result'].map({'BlockedShot':0, 'MissedShots':0, 'Goal':1, 'SavedShot': 0, 'ShotOnPost':0})
salah.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,minute,result,X,Y,xG,player,h_a,player_id,situation,year,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
22879,15217.0,92,0,0.823,0.369,0.062365,Mohamed Salah,h,1250,OpenPlay,2014,LeftFoot,4720,Chelsea,Swansea,4,2,2014-09-13 15:00:00,Eden Hazard,Pass
22880,20034.0,84,0,0.920,0.532,0.439938,Mohamed Salah,a,1250,OpenPlay,2014,LeftFoot,4560,Tottenham,Chelsea,5,3,2015-01-01 17:30:00,Eden Hazard,Pass
22881,46908.0,87,0,0.886,0.566,0.096760,Mohamed Salah,h,1250,OpenPlay,2014,LeftFoot,4929,Fiorentina,Atalanta,3,2,2015-02-08 11:30:00,Mario Gomez,BallTouch
22882,47192.0,0,0,0.964,0.688,0.049131,Mohamed Salah,a,1250,OpenPlay,2014,LeftFoot,4937,Sassuolo,Fiorentina,1,3,2015-02-14 17:00:00,David Pizarro,Chipped
22883,47197.0,16,0,0.900,0.677,0.060320,Mohamed Salah,a,1250,OpenPlay,2014,LeftFoot,4937,Sassuolo,Fiorentina,1,3,2015-02-14 17:00:00,Khouma Babacar,Pass


In [6]:
xG = salah['xG'].rolling(window=50).mean()
goals = salah['result'].rolling(window=50).mean()
# xG

In [7]:
diff = goals - xG
diff = diff.dropna()
# diff

In [8]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColorBar, ColumnDataSource
from bokeh.transform import linear_cmap
from bokeh.io import output_notebook
from bokeh.palettes import Spectral6, Magma6, Viridis6, Viridis256
from bokeh.io import export_png
output_notebook()

p1 = figure(title="xG diff SMA", background_fill_color = '#e5e5e5')

mapper = linear_cmap(field_name='y', palette=Viridis256 ,low=min(diff) ,high=max(diff))
source = ColumnDataSource(dict(x=range(len(diff)), y=diff))
p1.xaxis.axis_label = 'Shots'
p1.yaxis.axis_label = 'xG diff'
p1.circle(x='x', y='y', line_color=mapper, source=source, size=5)
# p1.line(range(len(diff)), diff, line_color='#c4c4c4')
show(p1)

# export_png(p1, filename="plot.png")

In [13]:
from bokeh.palettes import Spectral6, Viridis256
# print(Spectral6)
p1_line = figure(title="xG diff SMA", background_fill_color = '#e5e5e5')
p1_line.xaxis.axis_label = 'Shots'
p1_line.yaxis.axis_label = 'xG diff'
for i in range(len(diff)):
    p1_line.line([i, i+1], diff[i:i+2], line_color=f'{Viridis256[i//3]}')   ## We're choosing a color from the list according to the shot number, feel free to mess around with this. Check the documentation for Bokeh Palettes

show(p1_line)

In [10]:
xG_ema = salah['xG'].ewm(span=50, adjust=False).mean()
goals_ema = salah['result'].ewm(span=50, adjust=False).mean()
diff_ema = goals - xG
diff_ema = diff_ema.dropna()

In [11]:
p2 = figure(title="xG diff EMA", background_fill_color = '#202020')

mapper = linear_cmap(field_name='y', palette=Spectral6 ,low=min(diff_ema) ,high=max(diff_ema))
source = ColumnDataSource(dict(x=range(len(diff_ema)), y=diff_ema))

p2.circle(x='x', y='y', line_color=mapper, source=source)
p2.line(range(len(diff_ema)), diff_ema, line_color='#404040')
show(p2)

In [12]:
p3 = figure(title="xG diff SMA vs EMA", background_fill_color = '#202020')

p3.line(range(len(diff)), diff, line_color = 'blue', legend_label='SMA')
p3.line(range(len(diff_ema)), diff_ema, line_color = 'yellow', legend_label='EMA')
p1.legend.location = "top_right"
show(p3)

/usr/local/lib/python3.6/dist-packages/bokeh/models/plots.py:764: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
